In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import pylab
import plotly.express as px
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})

In [ ]:
IMG_H = 128
IMG_W = 128
IMG_C = 3

In [ ]:
def graph_scatter(x, colors):
    # choose a color palette with seaborn.
    num_classes = len(np.unique(colors))
    palette = np.array(sns.color_palette("hls", num_classes))

    # create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40, c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # add the labels for each digit corresponding to the label
    txts = []

    for i in range(num_classes):

        # Position of each label at median of data points.

        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

In [ ]:
def prep_stage(img):
    img= np.resize(img,(IMG_H,IMG_W,IMG_C))
    img = img.astype('float32')
    img /= 255
  
    return img.ravel()
def create_dataset_PIL(img_folder, limit=100, ext=".png"):
    img_data_array=[]
    class_name=[]
    
    for dir1 in os.listdir(img_folder):
        counter = 0
        if dir1 != ".DS_Store" and dir1 != ".ipynb_checkpoints":
            
            for file in os.listdir(os.path.join(img_folder, dir1)):

                image_path= os.path.join(img_folder, dir1,  file)
                _, file_extension = os.path.splitext(image_path)
                if file_extension == ext:
                    
                    image= np.array(Image.open(image_path))
                    
                    image = prep_stage(image)
                    
                    
                    img_data_array.append(image)
                    class_name.append(dir1)
                    counter += 1
                    if limit == counter:
                        break
                
    return np.array(img_data_array) , np.array(class_name)

In [ ]:
img_folder = "mura_data/RGB/mura_march_clean/test_data"
PIL_img_data, class_name=create_dataset_PIL(img_folder, 1000)

In [ ]:
print(len(PIL_img_data), len(class_name))

In [ ]:
labels = ["normal", "defect"]
target_dict={k: v for v, k in enumerate(labels)}
target_dict

In [ ]:
target_val =  [target_dict[class_name[i]] for i in range(len(class_name))]
len(target_val)
target_val = np.array(target_val)
target_val.shape

In [ ]:
PIL_img_data.shape

In [ ]:
PIL_img_data

In [ ]:
from sklearn.manifold import TSNE
import time
time_start = time.time()
RS = 123

tsne_result = TSNE(n_components=3, random_state=RS).fit_transform(PIL_img_data)
    
print(f"t-SNE done! Time elapsed: {time.time()-time_start} seconds")

In [ ]:
print(tsne_result.shape)

fig = px.scatter_3d(
    tsne_result, x=0, y=1, z=2,
    color=target_val, labels={'color': 'classes'}
)
fig.update_traces(marker_size=8)
# fig.show()
fig.write_image("fig1.png")

In [ ]:
# graph_scatter(tsne_result, target_val)